## Processamento de dados intermediários de glicose

O objetivo desse notebook é realizar um processamento das tabelas originais do MIMIC-IV e de algumas tabelas intermediárias com objetivo de gerar tabelas contendo mais dados úteis.

#### Informaçoes dos pacientes que fizeram exames de glicose

A tabela gerada (`glucose_norm.csv`) possui o identificador do paciente que realizou o exame na coluna `subject_id`. Utilizando essa informação podemos pegar as informações desses pacientes. As informações dos pacientes do MIMIC-IV estão disponíveis na tabela `omr.csv`, que possui informações sobre peso, altura, IMC e pressão arterial.

O código abaixo obtêm as informações de todos os pacientes que fizeram algum exame de glicose, e salva na tabela `patients_with_glucose.csv`

In [11]:
import pandas as pd

In [12]:
patient_info_path = "../data/external/omr.csv"
glucose_events_path = "../data/processed/glucose_norm.csv"
output_patients_with_glucose = "../data/interim/patients_with_glucose.csv"

In [13]:
df_glucose = pd.read_csv(glucose_events_path)
all_patients = df_glucose['subject_id']

df_patients = pd.read_csv(patient_info_path)
df_patients_with_glucose = df_patients[df_patients['subject_id'].isin(all_patients)]

print("Saving processed table in:", output_patients_with_glucose)
df_patients_with_glucose.to_csv(output_patients_with_glucose)

Saving processed table in: patients_with_glucose.csv


In [14]:
#visualizando a tabela gerada
df = pd.read_csv(output_patients_with_glucose)
print(df.head())
print("LEN:", len(df))

   Unnamed: 0  subject_id   chartdate  seq_num      result_name result_value
0           0    10000032  2180-04-27        1   Blood Pressure       110/65
1           1    10000032  2180-04-27        1     Weight (Lbs)           94
2           2    10000032  2180-05-07        1      BMI (kg/m2)         18.0
3           3    10000032  2180-05-07        1  Height (Inches)           60
4           4    10000032  2180-05-07        1     Weight (Lbs)        92.15
LEN: 1771620


#### Informações sobre aplicações de insulina em pacientes com glicose

Vamos novamente analisar pacientes que receberam aplicação de insulina, mas agora considerando pacientes que fizeram exame de glicose ao invés de CK (CPK). Para tanto, vamos usar os mesmos IDs analisados anteriormente.

A partir dessa tabela é gerada uma nova tabela com o nome de `insulin_inputs_in_glucose_patients.csv` com todos os eventos de aplicação de insulina em pacientes que fizeram algum exame de glicose selecionado anteriormente.

Novamente essa tabela (`inputevents.csv`) é muito grande (2,3GB) assim também foi processada em pedaços (chunks).

In [15]:
input_events_path = "../data/external/inputevents.csv"
insulin_inputs_output = "../data/interim/insulin_inputs_in_glucose_pacients.csv"

insulin_idxs = [223257, 223258, 223259, 223260, 223261, 223262, 229299, 229619]

In [16]:
chunksize = 10 ** 6

df_glucose = pd.read_csv(glucose_events_path)
all_patients = df_glucose['subject_id']

df_final = None
with pd.read_csv(input_events_path, chunksize=chunksize) as reader:
    for chunk in reader:
        df_patients_with_glucose = chunk[chunk['subject_id'].isin(all_patients)]
        df_insulin_inputs = df_patients_with_glucose[df_patients_with_glucose['itemid'].isin(insulin_idxs)]
        if len(chunk) > 0:
            if df_final is None:
                df_final = df_insulin_inputs
            else:
                df_final = pd.concat([df_final,df_insulin_inputs])


print("Saving processed table in:", insulin_inputs_output)
df_final.to_csv(insulin_inputs_output)

Saving processed table in: insulin_inputs_in_glucose_pacients.csv


In [17]:
#visualizando a tabela gerada
df = pd.read_csv(insulin_inputs_output)
print(df.head())
print("LEN:", len(df))

   Unnamed: 0  subject_id   hadm_id   stay_id  caregiver_id  \
0         360    10002013  23581541  39060235          6768   
1         362    10002013  23581541  39060235          6768   
2         374    10002013  23581541  39060235          6768   
3         376    10002013  23581541  39060235          6768   
4         379    10002013  23581541  39060235          6768   

             starttime              endtime            storetime  itemid  \
0  2160-05-18 22:00:00  2160-05-18 23:00:00  2160-05-19 00:07:00  223258   
1  2160-05-18 23:00:00  2160-05-19 01:09:00  2160-05-19 00:07:00  223258   
2  2160-05-19 01:09:00  2160-05-19 02:17:00  2160-05-19 01:09:00  223258   
3  2160-05-19 02:17:00  2160-05-19 06:25:00  2160-05-19 02:17:00  223258   
4  2160-05-19 05:33:00  2160-05-19 05:34:00  2160-05-19 05:33:00  223260   

      amount  ... ordercomponenttypedescription  ordercategorydescription  \
0   7.989103  ...          Main order parameter            Continuous Med   
1  12.8896